# ML Programmentwurf

> **Algorythmus**: Klassifikation
>
> **Datensatz**: Iris
>
> **Autor**: Oliver Seider


## Dataset Einlesen

Der Datensatz wird mit Pandas eingelesen. Es werden zusätzlich Namen für die Spalten definiert, obwohl diese ab dem nächten Schritt nicht mehr von Relevanz sind.

Anschließend wird der Datensatz mit Hilfe von *numpy* in ein zweidimensionales Array umgewandelt.

In [ ]:
import pandas as pd


headers = ['sepal length', 'sepal width', 'petal length', 'petal width', 'category']
pandas_dataframe = pd.read_csv('iris.data', header=None, names=headers)

array_2d = pandas_dataframe.to_numpy()

# print(pandas_dataframe)
# print(array_2d)
print('There are {0} samples in the dataset'.format(len(array_2d)))

## Umformen der Daten
Das Zweidimensionale Daten-Array wird in einem nächten Schritt in Features und Kategorie aufgeteilt.

In [ ]:
data_array_2d = array_2d[:150,:4]
category_array = array_2d[:150, 4:].reshape([150])

# print(data_array_2d)
# print(category_array)

## Aufteilen der Daten in Lern- und Testdaten
Der Test-Datensatz stellt 25 Prozent der Originaldaten dar. Bei der Aufspaltung wurde ein Zufallskoeffizient von 0 gewählt, was bedeutet dass immer die selbe Aufteilung ausgegeben wird.

In [ ]:
from sklearn.model_selection import train_test_split

reshaped_data_train, reshaped_data_test, target_data_train, target_data_test = train_test_split(data_array_2d, category_array, test_size=0.25, random_state=0)

## Erstellen des Modells
In dem nächsten Schritt wird die ***Support Vector Classification*** von *sklearn* eingesetzt, um ein Modell anhand der Lerndaten zu erstellen. 
Dafür wird die ***Radial Basis Function*** (kurz: *rbf*) als Kernel eingesetzt.

Der 'probability=True' Parameter ermöglicht es, die Wahrscheinlichkeiten mit denen eine Klasse erkannt wurde durch eine interne *5-fold cross-validation* auszugeben.

In [ ]:
from sklearn import svm

# Definition des Modells
classifier = svm.SVC(kernel='rbf', probability=True)

# Einlernen der Daten
classifier.fit(reshaped_data_train, target_data_train)

## Testen des Modells
Nun wird Anhand des Modells für die Testdaten entschieden, zu welcher Klasse diese gehören.
Die Variable *p_y* gibt die Wahscheinlichkeit p, mit welcher ein Datensatz zu einer jeweiligen Klasse angehört.

In [ ]:
# Klassen-Angehörigkeit von jedem Datensatatz nach dem Modell
y = classifier.predict(reshaped_data_test)

# Wahscheinlichkeit, mit welcher ein Datensatz zu den jeweiligen Klassen angehört
p_y = classifier.predict_proba(reshaped_data_test)


# print(y)
# print(p_y)

## Evaluierung des Modells
In diesem letzten Schritt werden die Ergebnisse aus dem Vorherigen Schritt anhand von einem *Classification Report* und einer *Confusion Matrix* evaluiert.

**Classification Report:**
- *Precision*: Gibt an, wie viele der identifizierten Elemente *true positives* sind (true positives / [true positives + false positives]).
- *Recall*: Gibt an, wie viele Elemente, die einer Klasse angehören, dieser auch zugeteilt wurden (true positives / [true positives + false negatives])
- *F1-Score*: Harmonischer Mittelwert aus *Precision* und *Recall*
- *Support*: Anzahl an Datensätzen, die zum Bestimmen der vorherigen Werte verwendet wurden.

**Confusion Matrix:**
Zweidimensionale Matrix, die für jede Klasse und deren Datensätze zeigt, wie häufig das Modell einen Datensatz einer Klasse zugeordnet hat. 
Dabei stellt jede Spalte eine Klasse und dessen Datensätze dar, während die Spalten angeben wie oft eine Datensatz dieser Klasse zugeordnet wurde.

In [ ]:
from sklearn import metrics


print('Classification Report:')
print(metrics.classification_report(target_data_test, y))

print('Confusion Matrix:')
print(metrics.confusion_matrix(target_data_test, y))